In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import data
from skimage.color import label2rgb
from skimage.filters import sobel
from skimage.measure import label
from skimage.segmentation import expand_labels, watershed

coins = data.coins()

# Make segmentation using edge-detection and watershed.
edges = sobel(coins)

# Identify some background and foreground pixels from the intensity values.
# These pixels are used as seeds for watershed.
markers = np.zeros(coins.shape, dtype=np.uint)
foreground, background = 1, 2
markers[130,150]=1
markers[130,200]=1

markers[0,0]=2

# based on https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_expand_labels.html#sphx-glr-auto-examples-segmentation-plot-expand-labels-py
ws = watershed(edges, markers)
seg1 = label(ws == foreground)

expanded = expand_labels(seg1, distance=10)

# Show the segmentations.
fig, axes = plt.subplots(nrows=1, ncols=2)

axes[0].imshow(coins, cmap="Greys_r")
axes[0].set_title("Original")

color1 = label2rgb(seg1, image=coins, bg_label=0)
axes[1].imshow(color1)
axes[1].set_title("Sobel+Watershed")


for a in axes:
    a.axis("off")
fig.tight_layout()
plt.show()

from jupyter_compare_view import compare
compare(coins, markers)

In [ ]:
import ipyreact
from traitlets import Unicode , Int, observe
from IPython.display import display


def is_prime_number(n):
    print(n)


class TldrawWidget(ipyreact.ReactWidget):
    my_text = Unicode("Hello World").tag(sync=True)
    my_text_out = Unicode("Hello World").tag(sync=True)
    base64img = Unicode("").tag(sync=True)
    image_width = Int(300).tag(sync=True)
    image_height = Int(100).tag(sync=True)

    @observe("my_text_out")
    def _observe_count(self, change):
        is_prime_number(self.my_text_out)

    _esm = """
    import { TDAssetType, TDShapeType, Tldraw, TldrawApp } from "@tldraw/tldraw";
    import * as React from "react";

    export default function App({image_width, image_height, my_text, base64img, set_my_text_out }) {
      const [app, setApp] = React.useState();

      const handleMount = React.useCallback((app: Tldraw) => {
        setApp(app);
      }, []);

      React.useEffect(() => {
        if (app) {
        app.patchAssets({
          myAssetId: {
            id: "myAssetId",
            type: TDAssetType.Image,
            size: [image_width, image_height],
            fileName: "card-repo.png",
            src: base64img
          }
        });

        app.createShapes({
          id: "myImage",
          type: TDShapeType.Image,
          assetId: "myAssetId",
          point: [0, 60],
          size: [image_width, image_height],
          isLocked: true
        });
        }
      }, [base64img, app]);

      const handleChange = (e) => {
        let my_current_stroke = e.selectedIds[0];
        let my_object = e.getShape(my_current_stroke);

        if (my_object === undefined) {
          console.log("no object selected");
        }
        if (my_object !== undefined) {
          let my_points = my_object.points;
          if (my_points !== undefined){
            console.log(my_object.id);
            console.log(my_points.length);
            console.log(my_points[my_points.length - 1]);
            set_my_text_out(my_object.point.toString())
          }
        }
      };

      return (
        <div
          style={{
            position: "relative",
            width: image_width,
            height: image_height+120,
          }}
        >
          <Tldraw onMount={handleMount} onChange={handleChange} />
        </div>
      );
    }

    """


import base64
import matplotlib.pyplot as plt
import io

fig, ax = plt.subplots()

ax.axis('off')
ax.imshow(coins, cmap='gray')

plt.tight_layout()
buf = io.BytesIO()
fig.savefig(buf, format='png',bbox_inches='tight', transparent="True", pad_inches=0)

plt.close()
buf.seek(0)
base64_img_string_pre = base64.b64encode(buf.getvalue()).decode()


base64_img_string = f"data:image/png;base64,{base64_img_string_pre}"

# base64 string of the PNG image
# b64_string = 'iVBORw0KGgoAAAANSUhEUgAAAAUAAAAFCAYAAACNbyblAAAAHElEQVQI12P4//8/w38GIAXDIBKE0DHxgljNBAAO9TXL0Y4OHwAAAABJRU5ErkJggg==' # for debugging
b64_string = base64_img_string_pre

# decode base64 string to bytes
decoded_bytes = base64.b64decode(b64_string)

# check if the file has the PNG signature
if decoded_bytes[:8] != b'\x89PNG\r\n\x1a\n':
    raise ValueError('Invalid PNG file')

# extract the IHDR chunk
ihdr_start = 8
ihdr_end = decoded_bytes.find(b'IHDR') + 4 + 8
ihdr_chunk = decoded_bytes[ihdr_start:ihdr_end]

# extract image width and height from the IHDR chunk
image_width = int.from_bytes(ihdr_chunk[8:12], byteorder='big')
image_height = int.from_bytes(ihdr_chunk[12:16], byteorder='big')

tldraw = TldrawWidget(image_width = image_width, image_height = image_height, base64img = base64_img_string)
display(tldraw)

In [ ]:

coord = tldraw.my_text_out.split(",")

coord = [int(float(i)) for i in coord]

coord[1] = coord[1] - 60

coord = [coord[1], coord[0]]

coord[0] = int(coord[0] / 1.43)
coord[1] = int(coord[1] / 1.43)
# print(coord)

markers = np.zeros(coins.shape, dtype=np.uint)
foreground, background = 1, 2
markers[coins < 30.0] = background
markers[coord[0], coord[1]]=foreground
# compare(coins, markers)

# based on https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_expand_labels.html#sphx-glr-auto-examples-segmentation-plot-expand-labels-py
ws = watershed(edges, markers)
seg1 = label(ws == foreground)

expanded = expand_labels(seg1, distance=10)

# Show the segmentations.
fig, ax = plt.subplots()
color1 = label2rgb(seg1, image=coins, bg_label=0)

ax.axis('off')

ax.imshow(color1)

plt.tight_layout()
buf = io.BytesIO()
fig.savefig(buf, format='png',bbox_inches='tight', transparent="True", pad_inches=0)
plt.close()
buf.seek(0)
base64_img_string_pre = base64.b64encode(buf.getvalue()).decode()


base64_img_string = f"data:image/png;base64,{base64_img_string_pre}"
tldraw.base64img = base64_img_string